In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import os
import matplotlib.image as img
import tqdm
from vae import Basic_VAE

#### Create Dataset


In [4]:
cats = []
directory = "data/cats"
count = 0
for catpic in os.listdir(directory):
    if count < 1000:
        # read from image and convert to tensor
        im = torch.tensor(img.imread(os.path.join(directory, catpic))).float()
        # permute to (channels, height, width) for conv2d layer
        im = torch.permute(im, (2, 0, 1))
        # normalize to range between -1 and 1
        im = im / 128 - 1
        cats.append(im)
        count+=1
    else:
        break
cats = torch.stack(cats)
print(cats.shape)

torch.Size([1000, 3, 64, 64])


#### Test/Training/Validation Split


In [5]:
# Split the data into what we use for testing and not testing
traing_test_split = 0.8
training_test_cutoff = int(cats.shape[0] * traing_test_split + 1)
random_perm = torch.randperm(cats.shape[0])
not_test_tensor = cats[:training_test_cutoff]
testing_tensor = cats[training_test_cutoff:]
# Split the data into what we use for training and cross validation
training_cv_split = 0.8
training_cv_cutoff = int(not_test_tensor.shape[0] * training_cv_split)
training_tensor = not_test_tensor[:training_cv_cutoff]
cv_tensor = not_test_tensor[training_cv_cutoff:]
print(training_tensor.shape)
print(cv_tensor.shape)
print(testing_tensor.shape)

torch.Size([640, 3, 64, 64])
torch.Size([161, 3, 64, 64])
torch.Size([199, 3, 64, 64])


#### Choose Hyperparameters and Build Model


In [6]:
from vae import Basic_VAE
hidden_dims = [16, 32, 64, 128]
latent_dim = 64
model = Basic_VAE(64, hidden_dims, latent_dim)

[128, 64, 32, 16]


In [7]:
# Testing the model
model.forward(training_tensor)

RuntimeError: Given groups=1, weight of size [16, 64, 5, 5], expected input[640, 3, 64, 64] to have 64 channels, but got 3 channels instead

#### Implement Loss Function

In [ ]:
def loss_function(self, reconstructed_img, input_img, mu, log_var, kld_weight=2):
    img_loss = F.mse_loss(reconstructed_img, input_img)
    # article on calculating kl divergence between 2 gaussians:
    # https://medium.com/@outerrencedl/variational-autoencoder-and-a-bit-kl-divergence-with-pytorch-ce04fd55d0d7
    kld_loss = torch.mean(
        torch.sum(-log_var + (log_var.exp() ** 2 + mu**2) / 2 - 1 / 2)
    )
    kld_loss *= kld_weight

    return img_loss + kld_loss

#### Implement Gradient Descent


In [ ]:
def gradient_descent(model, loss_func, x, y, xvalid, yvalid, lr=0.1, steps=5000):
    optimizer = optim.AdamW(model.parameters(), lr)

    losses = []
    valid_losses = []
    for _ in tqdm.trange(steps):
        model.train()
        loss = loss_func(model(x), y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        model.eval()
        valid_loss = loss_func(model(xvalid), yvalid)
        losses.append(loss.detach().numpy())
        valid_losses.append(valid_loss.detach().numpy())

    print(f"Final training loss: {losses[-1]}")

    return losses, valid_losses